# 1. Setup and imports

## 1.1 Imports

In [ ]:
import shutil
import torch

from IPython import display
from pathlib import Path

print(f"GPU device is available : {torch.cuda.is_available()}")
torch_device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

## 1.2 Setup variables + installing missing libraries for Colab

In [ ]:
try:
    from google.colab import drive
    # Mount drive to access to data
    drive.mount('/content/drive')
    import sys
    !mkdir /content/configs
    !cp /content/drive/MyDrive/Colab_notebooks/Master_thesis/code_base/configs_template/model_config_template.yaml /content/configs/model_config.yaml
    !cp /content/drive/MyDrive/Colab_notebooks/Master_thesis/code_base/configs_template/full_config_template.yaml /content/configs/full_config.yaml
    !cp /content/drive/MyDrive/Colab_notebooks/Master_thesis/code_base/configs_template/optimization_config_template.py /content/configs/optimization_config.py
    !cp -r /content/drive/MyDrive/Colab_notebooks/Master_thesis/code_base/lt_lib/ /content/lt_lib/
    sys.path.append('/content/lt_lib/')
    !pip install loguru
    !pip install ruamel.yaml
    !pip install wandb
    !pip install ray==2.9.3
    !pip install optuna
    display.clear_output()
    IN_COLAB = True
except:
    IN_COLAB = False

if not IN_COLAB:
  try:
        import kaggle
  except OSError:
        import sys
        sys.path.insert(1,'/kaggle/input/code-base/code_base/lt_lib/')
        !mkdir /kaggle/working/configs
        !cp /kaggle/input/configs/model_config.yaml /kaggle/working/configs/model_config.yaml
        !cp /kaggle/input/configs/full_config.yaml /kaggle/working/configs/full_config.yaml
        !cp /kaggle/input/configs/optimization_config.py /kaggle/working/configs/optimization_config.py
        IN_KAGGLE = True
  except:
        IN_KAGGLE = False
else:
    IN_KAGGLE = False

if IN_COLAB:
    !unzip /content/drive/MyDrive/data/dataset_v1.zip -d .
    !rm -rf __MACOSX/
    display.clear_output()
    INPUTS_DIR = "/content/dataset_v1"
    OUTPUTS_DIR = "/content/outputs_optim"
    # To verify that there is an access to the GPU
    !nvidia-smi
elif IN_KAGGLE:
    INPUTS_DIR = "/kaggle/input/dataset-v1/dataset_v1"
    OUTPUTS_DIR = "/kaggle/working/outputs_optim"
    # To verify that there is an access to the GPU
    !nvidia-smi 
else:
    INPUTS_DIR = "/content/datasets/dataset_test"
    OUTPUTS_DIR = "/content/outputs/outputs_test_optim"

import wandb
from lt_lib.entrypoints.optimization import optimization, OptimizationCliArgs

## 1.3 Setup wandb

In [ ]:
if IN_COLAB:
    !wandb login
if IN_KAGGLE:
    wandb.login(key)

# 2. Mono-run

In [ ]:
if IN_COLAB:
    !mkdir ray_results
    !unzip /content/drive/MyDrive/data/experiment-seed0_2024-04-09_15-59-06.zip -d ray_results/experiment-seed0_2024-04-09_15-59-06

In [ ]:
if IN_COLAB:
    !rm -r /content/ray_results/experiment-seed0_2024-04-09_15-59-06/trial_2024-04-09_16-33-27_5d8b84b4

In [ ]:
USE_WANDB = False

args = OptimizationCliArgs(
    inputs_directory=INPUTS_DIR,
    outputs_directory=OUTPUTS_DIR,
    model_config_path="/content/configs/model_config.yaml",
    config_path="/content/configs/train_config.yaml",
    optimization_config_path="/content/configs/optimization_config.py",
    use_wandb_ray_integration=USE_WANDB,
    restore_dir_path="",
    console_log_level="DEBUG",
)

optimization(args)

In [ ]:
if IN_COLAB:
    !zip -r /content/ray_results.zip ray_results/

In [ ]:
if IN_COLAB:
    !rm -r /content/outputs_optim
    !rm -r /content/ray_results.zip
    !rm -r /content/ray_results

# 3. Many-runs
## 3.1 Duplicate and modify config files

In [ ]:
NB_CONFIG = 10
CONFIG_PATH = Path("/content/configs/optimization_config.py")
MODIFY_PARAM = True

for k in range(NB_CONFIG):
    config_copy_path = CONFIG_PATH.parent / f"{CONFIG_PATH.name.split('.')[0]}-{k}{CONFIG_PATH.suffix}"
    shutil.copy2(CONFIG_PATH, config_copy_path)
    
    if MODIFY_PARAM:
        file_content = ""
        with open(config_copy_path, "r") as f:
            file_content = f.read()
        file_content = file_content.replace("random_seed = 0", f"random_seed = {k}")
        with open(config_copy_path, "w") as f:
            f.write(file_content)

In [ ]:
FILES_ROOT_DIR = Path("/content/configs")
MODIFY_CONFIGS = False

if MODIFY_CONFIGS:
    for file_path in FILES_ROOT_DIR.glob("*optimization*"):
        file_content = ""
        with open(file_path, "r") as f:
            file_content = f.read()
        file_content = file_content.replace("num_samples=5", "num_samples=10")
        with open(file_path, "w") as f:
            f.write(file_content)

## 3.2 Many-optimizations

In [ ]:
USE_WANDB = False
NB_CONFIG = 10

for k in range(NB_CONFIG):
    args = OptimizationCliArgs(
        inputs_directory=INPUTS_DIR,
        outputs_directory=OUTPUTS_DIR,
        model_config_path=f"/content/configs/model_config.yaml",
        config_path=f"/content/configs/train_config.yaml",
        optimization_config_path=f"/content/configs/optimization_config-{k}.py",
        use_wandb_ray_integration=USE_WANDB,
        restore_dir_path="",
        console_log_level="WARNING",
    )

    optimization(args)

In [ ]:
if IN_COLAB:
    !zip -r /content/ray_results.zip ray_results

In [ ]:
if IN_COLAB:
    !rm -r /content/outputs_optim
    !rm -r /content/ray_results.zip
    !rm -r /content/ray_results